In [1]:
import pandas as pd
import time
import numpy as np
import requests
import psycopg2
import json
import simplejson
import urllib
import config
import ast
import bs4
from pymongo import MongoClient
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup as BS
from operator import itemgetter
from sklearn.cluster import KMeans
from sqlalchemy import create_engine


In [2]:
conn_str = "dbname='travel_with_friends' user='Gon' host='localhost'"
# conn_str = "dbname='travel_with_friends' user='Zoesh' host='localhost'"

In [3]:
conn = psycopg2.connect(conn_str)   
cur = conn.cursor()   
# cur.execute("select index, name, coord0, coord1 from poi_detail_table where city !='%s' and state = '%s';" %(current_city, current_state))
cur.execute("select distinct city, state from poi_detail_table;" )
all_cities = cur.fetchall()

In [4]:
all_cities[0][0]

'Victoria'

In [5]:
cities_coords = pd.read_csv('cities_coords.csv', header=None)
cities_coords.columns = ['area_code', 'city','state','nation', 'coord0','coord1']
cities_coords = cities_coords[['city','state','nation', 'coord0','coord1']].drop_duplicates()
cities_coords.reset_index(drop = True, inplace = True)

In [6]:
geolocator = Nominatim()

for items in all_cities:
    if cities_coords[cities_coords['state'] == items[1]][cities_coords.city == items[0]].shape[0] == 0:
        location_name = ', '.join([items[0], items[1]])
        try:
            location = geolocator.geocode(location_name)
            cities_coords.loc[len(cities_coords)] = [items[0], items[1], 'US', location.latitude, location.longitude]
        except:
            "error, rest"
            time.sleep(20)
            print" start again"
            
            
#         print cities_coords.loc(len(cities_coords))

/Users/Gon/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


 start again
 start again
 start again
 start again


In [7]:
cities_coords.to_csv('all_cities_coords.csv')

In [8]:
engine = create_engine('postgresql://Gon@localhost:5432/travel_with_friends')
cities_coords.to_sql('all_cities_coords',engine, if_exists = "replace")

In [9]:
client = MongoClient()
db = client.zoeshrm
db.TripAdvisor_state_park.count()

2647

In [11]:
import us_state_abbrevation as abb
state_abb_dict = abb.state2abb
# state_abb['California']
print state_abb_dict.keys()[state_abb_dict.values().index('CA')]


California


In [ ]:
poi_detail_state_park=pd.DataFrame(columns=['index','name','street_address','city','state_abb','state','postal_code','country','address','coord_lat','coord_long','num_reviews','review_score','ranking','tag','visit_length','fee','description'])

In [132]:
state_park_pages = db.TripAdvisor_state_park.find()
index = 0
for page in state_park_pages[len(poi_detail_state_park):]:
    s = BS(page['html'], "html.parser")
    #index
    #name
    
    input_list = []
    print name
    name = s.find('h1', attrs = {'class':'heading_name'}).text.strip()
    
    #street_address
    street_address = s.find('span', attrs = {'class':'street-address'}).text.strip()
    #city
    city = s.find('span', attrs = {'property':'addressLocality'}).text.strip()
    
    #state
    state_abb = s.find('span', attrs = {'property':'addressRegion'}).text.strip()
    if state_abb:
        try:
            state = state_abb_dict.keys()[state_abb_dict.values().index(state_abb)]
        except:
            print name, "not state_abb or not USA state"
            state = state_abb
    else:
        state_abb = None
        state = None
    #postal_code
    postal_code = s.find('span', attrs = {'property':'postalCode'}).text.strip()
    #country
    if s.find('span', attrs = {'property':'addressCountry'}).get('content'):
        country = s.find('span',{'property':'addressCountry'}).get('content')
    elif s.find('span',{'property':'addressCountry'}).get('content') == None:
        country = s.find('span',{'property':'addressCountry'}).text.strip()
    else:
        country = 'United States'
    #address
    if state_abb:
        full_address = street_address+', '+city+', '+state_abb+', '+postal_code+', '+country
    else:
        print name, "No State, maybe not in United States"
        full_address = street_address+', '+city+', '+postal_code+', '+country

#     try:
#         location =geolocator.geocode(street_address+', '+city+', '+state_abb)
#     except:
#         "error, rest"
#         time.sleep(20)
#         print" start again"
#     if location:
#         #coord_lat
#         poi_detail_state_park['coord_lat'] = location.latitude 
#         #coord_long
#         poi_detail_state_park['coord_long'] =location.longitude
#     else:
#         print" location not found: ", name, "address : ", full_address
    #num_reviews
    try:
        num_reviews = s.find('div', attrs = {'class': 'rs rating'}).find('a').get('content')
    except:
        num_reviews = 0
        print name ," num_reviews not found"
    #review_score
    try:
         review_score = s.find('div', attrs = {'class': 'heading_rating separator'}).find('img').get('content')
    except:
        review_score = 0 
        print name ,"review_score not found"
    #ranking
    try:
        ranking = s.find('b', attrs = {'class':'rank_text wrap'}).text.strip().replace('#',"")
    except:
        ranking = 0
        print name ,"ranking not found"
    #tag
    try:
        tags = ", ".join(label.text.strip() for label in s.select('div.detail > a') + s.select('span[class="collapse hidden"] > a'))
    except:
        tags = None
        print name ,"tag not found"
    #visit_length
    if s.find(text ="Recommended length of visit:"):
        visit_length = s.find(text ="Recommended length of visit:").parent.next_sibling
    else:
        visit_length = None
    #fee
    if s.find(text= "Fee:"):
        fee = s.find(text= "Fee:").parent.next_sibling.upper()
    else:
        fee = 'NO'
    #description
    if s.find('div', attrs = {'class': "listing_details"}):
        description = s.find('div', attrs = {'class': "listing_details"}).text.strip()
    else:
        description = None
    
    input_list = [index, name, street_address, city, state_abb, state, postal_code, country, full_address,None,None,num_reviews, review_score, ranking,tags ,visit_length ,fee , description]
    
    poi_detail_state_park.loc[len(poi_detail_state_park)] = input_list
    index += 1
    

Skookumchuck Narrows Provincial Park
Skookumchuck Narrows Provincial Park
Old Corkscrew Golf Club
Fakahatchee Strand Preserve State Park & Boardwalk
Falling Waters State Park
Fanning Springs State Park
Faver-Dykes State Park
Florida Caverns State Park
Forest Capital Musem
Fort Clinch State Park
Fort Cooper State Park
Stephen Foster Folk Culture Center State Park
Fort George Island Cultural State Park
Fort Mose
Fort Pierce Inlet State Park
Fort Zachary Taylor Historic State Park
Fred Gannon Rocky Bayou State Park
Gainesville-Hawthorne State Trail
Gamble Plantation Historic State Park
Gamble Rogers Memorial State Recreation Area
Gasparilla Island State Park
George Crady Bridge Fishing Pier State Park
Grayton Beach State Park
Henderson Beach State Park
Highlands Hammock State Park
Hillsborough River State Park
Ellie Schiller Homosassa Springs Wildlife State Park
Honeymoon Island State Park
Hontoon Island State Park
Hugh Taylor Birch State Park
Ichetucknee Springs State Park
Indian Key Sta

In [134]:
poi_detail_state_park.drop_duplicates().shape

(2647, 18)

In [135]:
poi_detail_state_park.shape

(2647, 18)

In [139]:
poi_detail_state_park.to_csv("poi_detail_state_park.csv", encoding=('utf-8'))

In [140]:
poi_detail_state_park.to_sql('poi_detail_state_park_table',engine, if_exists = "replace")

In [55]:
tags = ", ".join(label.text for label in s.select('div.detail > a') + s.select('span[class="collapse hidden"] > a'))
tags

u'Historic Sites, Islands, Nature & Parks, Sights & Landmarks, National Parks, Nature & Wildlife Areas'

In [118]:
htmlurl = 'https://www.tripadvisor.com/Attraction_Review-g35805-d1134861-Reviews-Cloud_Gate-Chicago_Illinois.html'
htmlurl = 'https://www.tripadvisor.com/Attraction_Review-g60713-d127854-Reviews-San_Francisco_Zoo-San_Francisco_California.html'
htmlurl = 'https://www.tripadvisor.com/Attraction_Review-g60750-d104122-Reviews-San_Diego_Zoo-San_Diego_California.html'
htmlurl = 'https://www.tripadvisor.com/Attraction_Review-g60713-d102523-Reviews-Alcatraz_Island-San_Francisco_California.html'
htmlurl = 'https://www.tripadvisor.com/Attraction_Review-g32474-d4236729-Reviews-Harmony_Headlands_State_Park-Harmony_San_Luis_Obispo_County_California.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

r=requests.get(htmlurl,headers=headers)
s = BS(r.text, 'html.parser')


# for div in s.find('div', attrs = {'class' : "separator" }):
#     for tag in div.:
#         if tag.name == 'div' and tag.get('class', '') == ['detail']:
#             print tag.text
#     for item in div.contents:
# #         print item
#         if type(item)== 'bs4.element.Tag' and item.name == "detail":
#             print 1234567890
for div in s.findAll("div", {"class": "separator"}):
    for tag in div.contents:
        if isinstance(tag, bs4.element.Tag) and tag.get('class',"") == ['detail'] :
            tags =  tag.text.encode('utf8').strip()


In [101]:
s.find('span',{'property':'addressCountry'}).get('content')
# s.select('span[property="addressCountry"]').get('content')

AttributeError: 'list' object has no attribute 'get'

In [108]:
#index

#name
name = s.find('h1', attrs = {'class':'heading_name'}).text.strip()
#city
city = s.find('span', attrs = {'property':'addressLocality'}).text.strip()
street_address = s.find('span', attrs = {'class':'street-address'}).text.strip()
#state
state_abb = s.find('span', attrs = {'property':'addressRegion'}).text.strip()
# state = state_abb_dict.keys()[state_abb_dict.values().index(state_abb)]
postal_code = s.find('span', attrs = {'property':'postalCode'}).text.strip()

#country
country = s.find('span',{'property':'addressCountry'}).get('content')
#address
# full_address = street_address+', '+city+', '+state_abb+', '+postal_code+', '+country

# from geopy.geocoders import Nominatim
# geolocator = Nominatim()
# location =geolocator.geocode(street_address+', '+city+', '+state_abb+', '+country)
# #coord_lat
# coord_lat = location.latitude 
# #coord_long
# coord_long =location.longitude
#num_reviews
num_reviews = s.find('div', attrs = {'class': 'rs rating'}).find('a').get('content')

#review_score
review_score = s.find('div', attrs = {'class': 'heading_rating separator'}).find('img').get('content')

#ranking
ranking = s.find('b', attrs = {'class':'rank_text wrap'}).text.strip().replace('#',"")

#tag
tags = ", ".join(label.text for label in s.select('div.detail > a') + s.select('span[class="collapse hidden"] > a'))

#visit_length
# visit_length = s.find(text ="Recommended length of visit:").parent.next_sibling

# #fee
# fee = s.find(text= "Fee:").parent.next_sibling

#description
description = s.find('div', attrs = {'class': "listing_details"}).text.strip()


AttributeError: 'NoneType' object has no attribute 'text'

In [125]:
country = s.find('span',{'property':'addressCountry'}).text.strip()
country

u'Mexico'

In [109]:
<span class="country-name" property="addressCountry">Mexico</span>

<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<link href="https://static.tacdn.com/favicon.ico" id="favicon" rel="icon" type="image/x-icon"/>
<link color="#589442" href="https://static.tacdn.com/img2/icons/ta_square.svg" rel="mask-icon" sizes="any"/>
<script type="text/javascript">
   window.onerror = function onErrorFunc(msg, url, line, colno, error) {
if(!window.ta || !ta.has('ta.js_error_array.processed')) {
if(typeof js_error_array == 'undefined') {
js_error_array = [];
}
var err = error;
js_error_array[js_error_array.length] = {'msg': msg, 'error_script': url, 'line': line, 'column': colno, 'error': err, 'ready_state': document.readyState};
return true;
}
else {
if(window.ta && ta.util && ta.util.error && ta.util.error.record) {
ta.util.error.record(error, 'error post load:: ' + msg, null, {'error_script': url, 'line': line, 'column': c

In [ ]:
# s.select('div[class="detail"] > a')

In [ ]:
# <span class="collapse hidden">, <a href="/Attractions-g60713-Activities-c57-t68-San_Francisco_California.html">Nature &amp; Wildlife Areas</a></span>

In [ ]:
# detail = {}
# addition_info = s.find('div', attrs = {'class':'details_wrapper'}).text.strip('\n').replace("\n\n","\n").split('\n')
# # if addition_info[0] == 'Description':
# #     print addition_info[1]
# addition_info

# for info in addition_info:
#     info_list = info.split(':')
#     if info_list[0]=="Fee":
#         details["Fee"] = info_list[1]
#     else:
#         details["length of visit"] = info_list[1]
# details

In [ ]:
# fee = s.find('div', {'class':'details_wrapper'})
# fee
# length_visit = s.find(text ="Recommended length of visit:").parent.next_sibling
# length_visit
# fee = s.find(text= "Fee:").parent.next_sibling
# fee


In [ ]:
# description = s.find('div', attrs = {'class': "listing_details"}).text.strip()
# print description

In [ ]:
# print name, ranking, review_score, num_reviews,"tag:", tags, "add:",full_address, country